# 統計LODを色々試すnotebook
## プロパティパスを使ったSPQRQL、使わないSPARQL

SPARQL初心者の自分は、プロパティパスを使った場合エリアコードをキーに興味のある様々な値を複数のプロパティを使ってどのように取得すべきかわからないので、冗長ではあるけどエリアコードの変数を明示してプロパティパスを使わずサンプルを書いています。

In [1]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
# 新宿区の日本人の人口を求める（プロパティパス）
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#> 
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX cd-code:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/> 
PREFIX cd-attribute:<http://data.e-stat.go.jp/lod/ontology/crossDomain/attribute/> 
PREFIX cd-dimension-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/2016/> 
PREFIX cd-code-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/2016/> 
PREFIX pc-measure-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/measure/2010/> 
PREFIX pc-dimension-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/dimension/2010/> 
PREFIX pc-code-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/code/2010/> 
PREFIX sac:<http://data.e-stat.go.jp/lod/sac/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select  ?o 
where {
      ?s pc-measure-2010:population ?o ;
         cd-dimension:standardAreaCode / ic:市区町村 "新宿区"@ja ;
         cd-dimension-2016:sex cd-code-2016:sex-total ; 
         cd-dimension-2016:nationality cd-code-2016:nationality-japanese ; 
         pc-dimension-2010:area pc-code-2010:area-all ; 
         cd-dimension-2016:age cd-code-2016:age-total . 
}

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query

{'head': {'vars': ['o']},
 'results': {'bindings': [{'o': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
                                 'type': 'typed-literal',
                                 'value': '286398'}}]}}

In [5]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
# プロパティパスを使わないで書いてみる
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#> 
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX cd-code:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/> 
PREFIX cd-attribute:<http://data.e-stat.go.jp/lod/ontology/crossDomain/attribute/> 
PREFIX cd-dimension-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/2016/> 
PREFIX cd-code-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/2016/> 
PREFIX pc-measure-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/measure/2010/> 
PREFIX pc-dimension-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/dimension/2010/> 
PREFIX pc-code-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/code/2010/> 
PREFIX sac:<http://data.e-stat.go.jp/lod/sac/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select  ?o 
where {
      ?s pc-measure-2010:population ?o ;
         cd-dimension:standardAreaCode ?areacode.
      ?areacode  ic:市区町村  "新宿区"@ja.
      ?s cd-dimension-2016:nationality cd-code-2016:nationality-japanese ; 
         cd-dimension-2016:sex cd-code-2016:sex-total ; 
         pc-dimension-2010:area pc-code-2010:area-all ; 
         cd-dimension-2016:age cd-code-2016:age-total . 
}

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query

{'head': {'vars': ['o']},
 'results': {'bindings': [{'o': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
                                 'type': 'typed-literal',
                                 'value': '286398'}}]}}

# 統計LODサポートを使って興味のあるプロパティを検索し、統計LODのデータを取得する

## 保育園数や保育園入所待機児童数を取得してみます。
- [統計LODサポート](http://idease.info/lodstat/)に「保育」と入力して検索。
- 「保育所数」「保育所在所児数」「保育所入所待機児童数」などのプロパティが表示されます。この中からまず、保育所入所待機児童数を取得することにします。
- 統計LODサポートで検索された「保育所入所待機児童数」をクリックします。
- 保育所数のプロパティ（predicate）「http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/J250502」 が長いのでPREFIXを"ssds-measure-2016"のように設定します。
- 待機児童数（この場合RDFの目的語になる）を指す変数を「?taikijido」とします
- セルID　 保育所数の述語　　待機児童数; のRDFを記述
- セルID（省略形式）　エリアコード（PREFIXを利用）を指す述語 エリアコード（未知の変数）. を記述
- [共通語彙基盤](http://imi.go.jp/ns/core/Core232.html)のコア語彙「市区町村」を使って、エリアコード  市区町村 新宿区. を記述し、新宿区のエリアコードを取得します。
- SPARQLを実行すると新宿区の待機児童数が変数taikijidoとして取得されます。


In [6]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
#新宿区の待機児童数
PREFIX ssds-measure-2016:<http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/>
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select ?taikijido
where{
    ?s ssds-measure-2016:J250502 ?taikijido;
        cd-dimension:standardAreaCode ?areacode.
     ?areacode ic:市区町村  "新宿区"@ja.
}

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query

{'head': {'vars': ['taikijido']},
 'results': {'bindings': [{'taikijido': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
                                         'type': 'typed-literal',
                                         'value': '176'}}]}}

## UNIONを使って「新宿区」と「文京区」のOR条件で取得する

In [20]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
%display table
PREFIX ssds-measure-2016:<http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/>
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select ?保育所数
where{
        {
        ?s ssds-measure-2016:J2503 ?保育所数;
        cd-dimension:standardAreaCode / ic:市区町村 "新宿区"@ja. 
            } union {
        ?s ssds-measure-2016:J2503 ?保育所数;
        cd-dimension:standardAreaCode / ic:市区町村 "文京区"@ja. 
        }
}

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query
Display: table

保育所数
36
28


## 都道府県名のプロパティで東京都のエリアコードを取得し、東京都の待機児童数を取得

In [12]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
%display table
PREFIX ssds-measure-2016:<http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/>
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select ?待機児童数 ?市区町村
where{
    ?s ssds-measure-2016:J250502 ?待機児童数;
        cd-dimension:standardAreaCode ?areacode.
     ?areacode ic:都道府県  "東京都"@ja;
         ic:市区町村 ?市区町村.
}
limit 10

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query
Display: table

待機児童数,市区町村
0,新島村
249,調布市
1,瑞穂町
270,豊島区
438,大田区
52,清瀬市
0,奥多摩町
125,北区
257,町田市
47,狛江市


## 区ごとの「保育所在所児童数」「保育所数」を組み合わせてみます。
プロパティパスで逆パス等を使うとまとめて記述できそうだけど、、、、自分は書けないのでとりあえずシンプルに書いてみます。

In [16]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
%display table
PREFIX ssds-measure-2016:<http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/>
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select ?市区町村 ?保育所数  ?在所児童 ?待機児童
where{
    ?s1 ssds-measure-2016:J250502 ?待機児童;
        cd-dimension:standardAreaCode ?areacode.
     ?areacode ic:都道府県  "東京都"@ja;
         ic:市区町村 ?市区町村.
    ?s2 ssds-measure-2016:J2503 ?保育所数;
        cd-dimension:standardAreaCode ?areacode.
    ?s3 ssds-measure-2016:J2506 ?在所児童;
        cd-dimension:standardAreaCode ?areacode.
}

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query
Display: table

市区町村,保育所数,在所児童,待機児童
目黒区,26,2498,132
文京区,28,2327,96
新島村,3,63,0
八丈町,3,162,0
東大和市,15,1788,79
小金井市,13,1404,188
清瀬市,11,937,52
国立市,11,1151,32
渋谷区,29,2305,73
あきる野市,15,1779,35


さらに関係ありそうなデータをエリアコードをキーに追加していきたいと思います。
統計LODサポートで今度は「病院」を検索します。「一般病院数」のプロパティがあるのでこれをSPARQLに追加します。
また最初のサンプルで取得しているような、区毎の人口もざっくり（全年齢、全盛別の値で）追加してみます。

In [19]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
%display table
PREFIX ssds-measure-2016:<http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/>
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX pc-measure-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/measure/2010/> 
PREFIX pc-dimension-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/dimension/2010/> 
PREFIX cd-dimension-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/2016/> 
PREFIX pc-code-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/code/2010/> 
PREFIX cd-code-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/2016/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select ?siku ?hoikusyo  ?zaisyo ?taiki ?hospital ?jinko
where{
    ?s1 ssds-measure-2016:J250502 ?taiki;
        cd-dimension:standardAreaCode ?areacode.
     ?areacode ic:都道府県  "東京都"@ja;
         ic:市区町村 ?siku.
    ?s2 ssds-measure-2016:J2503 ?hoikusyo;
        cd-dimension:standardAreaCode ?areacode.
    ?s3 ssds-measure-2016:J2506 ?zaisyo;
        cd-dimension:standardAreaCode ?areacode.
    ?s4 ssds-measure-2016:I510120 ?hospital;
        cd-dimension:standardAreaCode ?areacode.
?s5 pc-measure-2010:population ?jinko;
        cd-dimension:standardAreaCode ?areacode;
        cd-dimension-2016:sex cd-code-2016:sex-total; 
         cd-dimension-2016:nationality cd-code-2016:nationality-japanese; 
         pc-dimension-2010:area pc-code-2010:area-all; 
         cd-dimension-2016:age cd-code-2016:age-total .
}

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query
Display: table

siku,hoikusyo,zaisyo,taiki,hospital,jinko
八王子市,82,9172,253,33,568293
稲城市,12,1424,50,3,83200
日の出町,4,504,26,2,16591
羽村市,12,1217,6,2,55379
瑞穂町,7,618,1,1,32979
利島村,1,24,0,0,339
目黒区,26,2498,132,9,260870
町田市,56,5257,257,14,419062
青梅市,32,3322,19,8,138341
杉並区,55,5390,285,17,506982
